# Kaggle Titanic 2eme essai !

## Jeu de test

In [6]:
import pandas as pd
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

train = pd.read_csv("../datasources/titanic/train.csv")
test = pd.read_csv("../datasources/titanic/test.csv")

### Travail sur le nom

In [44]:
familynames = []
for noms in test["Name"]:
    familynames.append(re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(1))
pdfamilynames = pd.DataFrame(familynames, columns = ['familynames'])

famsurv = test.join(pdfamilynames)
#famsurv['familynames'].value_counts()

## Création d'une fonction de préparation

In [30]:
def dataprep(data):
    sexe = pd.get_dummies(data['Sex'], prefix='sex')
    cabin = pd.get_dummies(data['Cabin'].fillna('X').str[0], prefix='Cabin')
    # Age 
    age = data['Age'].fillna(data['Age'].mean())
    enfantf = []
    for i in range(data.shape[0]) :
        if age[i] < 9:
            enfantf.append(0)
        else:
            enfantf.append(1)
    enfantd = pd.DataFrame(enfantf, columns = ['Enfant'])
    
    # Titre (Master principalement)
    ismaster = []
    for noms in data["Name"]:
        if (re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(2) == "Master") :
            ismaster.append(1)
        else:
            ismaster.append(0)
    pdismaster = pd.DataFrame(ismaster, columns = ['isMaster'])

    emb = pd.get_dummies(data['Embarked'], prefix='emb')
    # Prix du billet / Attention une donnée de test n'a pas de Prix !
    faresc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Fare']].fillna(0)), columns = ['Prix'])
    # Classe
    pc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Pclass']]), columns = ['Classe'])

    dp = data[['SibSp', 'Parch']].join(pc).join(sexe).join(emb).join(faresc).join(cabin).join(age).join(pdismaster).join(enfantd)
    return dp

In [31]:
Xtrain = dataprep(train)
# remove the Cabin with T value as this does not exist in the test dataset !
del Xtrain['Cabin_T']
Xtest = dataprep(test)

In [32]:
Xtrain.describe()

SibSp       Parch      Classe  sex_female    sex_male       emb_C  \
count  891.000000  891.000000  891.000000  891.000000  891.000000  891.000000   
mean     0.523008    0.381594    0.654321    0.352413    0.647587    0.188552   
std      1.102743    0.806057    0.418036    0.477990    0.477990    0.391372   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.500000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    1.000000    0.000000    1.000000    0.000000   
75%      1.000000    0.000000    1.000000    1.000000    1.000000    0.000000   
max      8.000000    6.000000    1.000000    1.000000    1.000000    1.000000   

            emb_Q       emb_S        Prix     Cabin_A     Cabin_B     Cabin_C  \
count  891.000000  891.000000  891.000000  891.000000  891.000000  891.000000   
mean     0.086420    0.722783    0.062858    0.016835    0.052750    0.066218   
std      0.281141    0.447876    0.096995    0.128725    0.223659    0.248802   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.015440    0.000000    0.000000    0.000000   
50%      0.000000    1.000000    0.028213    0.000000    0.000000    0.000000   
75%      0.000000    1.000000    0.060508    0.000000    0.000000    0.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

          Cabin_D     Cabin_E     Cabin_F     Cabin_G     Cabin_X         Age  \
count  891.000000  891.000000  891.000000  891.000000  891.000000  891.000000   
mean     0.037037    0.035915    0.014590    0.004489    0.771044   29.699118   
std      0.188959    0.186182    0.119973    0.066890    0.420397   13.002015   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.420000   
25%      0.000000    0.000000    0.000000    0.000000    1.000000   22.000000   
50%      0.000000    0.000000    0.000000    0.000000    1.000000   29.699118   
75%      0.000000    0.000000    0.000000    0.000000    1.000000   35.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000   80.000000   

         isMaster      Enfant  
count  891.000000  891.000000  
mean     0.044893    0.939394  
std      0.207186    0.238740  
min      0.000000    0.000000  
25%      0.000000    1.000000  
50%      0.000000    1.000000  
75%      0.000000    1.000000  
max      1.000000    1.000000

In [33]:
Xtrain.head(3)

SibSp  Parch  Classe  sex_female  sex_male  emb_C  emb_Q  emb_S      Prix  \
0      1      0     1.0           0         1      0      0      1  0.014151   
1      1      0     0.0           1         0      1      0      0  0.139136   
2      0      0     1.0           1         0      0      0      1  0.015469   

   Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_X  \
0        0        0        0        0        0        0        0        1   
1        0        0        1        0        0        0        0        0   
2        0        0        0        0        0        0        0        1   

    Age  isMaster  Enfant  
0  22.0         0       1  
1  38.0         0       1  
2  26.0         0       1

## Entrainement du modèle

In [34]:
y = train.Survived
clf = LinearSVC(random_state=0)
#rf = RandomForestClassifier(n_estimators=100, random_state=0, max_features=2)
clf.fit(Xtrain, y)
p_tr = clf.predict(Xtrain)
print ("Score Train -->", round(clf.score(Xtrain, y) *100,4), " %")

Score Train --> 82.4916  %


## Formattage des résultats pour Kaggle ;-)

In [35]:
Xtrain.head()

SibSp  Parch  Classe  sex_female  sex_male  emb_C  emb_Q  emb_S      Prix  \
0      1      0     1.0           0         1      0      0      1  0.014151   
1      1      0     0.0           1         0      1      0      0  0.139136   
2      0      0     1.0           1         0      0      0      1  0.015469   
3      1      0     0.0           1         0      0      0      1  0.103644   
4      0      0     1.0           0         1      0      0      1  0.015713   

   Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_X  \
0        0        0        0        0        0        0        0        1   
1        0        0        1        0        0        0        0        0   
2        0        0        0        0        0        0        0        1   
3        0        0        1        0        0        0        0        0   
4        0        0        0        0        0        0        0        1   

    Age  isMaster  Enfant  
0  22.0         0       1  
1  38.0         0       1  
2  26.0         0       1  
3  35.0         0       1  
4  35.0         0       1

In [36]:
Xtest.head()

SibSp  Parch  Classe  sex_female  sex_male  emb_C  emb_Q  emb_S      Prix  \
0      0      0     1.0           0         1      0      1      0  0.015282   
1      1      0     1.0           1         0      0      0      1  0.013663   
2      0      0     0.5           0         1      0      1      0  0.018909   
3      0      0     1.0           0         1      0      0      1  0.016908   
4      1      1     1.0           1         0      0      0      1  0.023984   

   Cabin_A  Cabin_B  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_X  \
0        0        0        0        0        0        0        0        1   
1        0        0        0        0        0        0        0        1   
2        0        0        0        0        0        0        0        1   
3        0        0        0        0        0        0        0        1   
4        0        0        0        0        0        0        0        1   

    Age  isMaster  Enfant  
0  34.5         0       1  
1  47.0         0       1  
2  62.0         0       1  
3  27.0         0       1  
4  22.0         0       1

In [37]:
Xtest.describe()

SibSp       Parch      Classe  sex_female    sex_male       emb_C  \
count  418.000000  418.000000  418.000000  418.000000  418.000000  418.000000   
mean     0.447368    0.392344    0.632775    0.363636    0.636364    0.244019   
std      0.896760    0.981429    0.420919    0.481622    0.481622    0.430019   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    1.000000    0.000000    1.000000    0.000000   
75%      1.000000    0.000000    1.000000    1.000000    1.000000    0.000000   
max      8.000000    9.000000    1.000000    1.000000    1.000000    1.000000   

            emb_Q       emb_S        Prix     Cabin_A     Cabin_B     Cabin_C  \
count  418.000000  418.000000  418.000000  418.000000  418.000000  418.000000   
mean     0.110048    0.645933    0.069373    0.016746    0.043062    0.083732   
std      0.313324    0.478803    0.109046    0.128474    0.203240    0.277317   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.015412    0.000000    0.000000    0.000000   
50%      0.000000    1.000000    0.028213    0.000000    0.000000    0.000000   
75%      0.000000    1.000000    0.061429    0.000000    0.000000    0.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

          Cabin_D     Cabin_E     Cabin_F     Cabin_G     Cabin_X         Age  \
count  418.000000  418.000000  418.000000  418.000000  418.000000  418.000000   
mean     0.031100    0.021531    0.019139    0.002392    0.782297   30.272590   
std      0.173797    0.145320    0.137177    0.048912    0.413179   12.634534   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.170000   
25%      0.000000    0.000000    0.000000    0.000000    1.000000   23.000000   
50%      0.000000    0.000000    0.000000    0.000000    1.000000   30.272590   
75%      0.000000    0.000000    0.000000    0.000000    1.000000   35.750000   
max      1.000000    1.000000    1.000000    1.000000    1.000000   76.000000   

         isMaster      Enfant  
count  418.000000  418.000000  
mean     0.050239    0.956938  
std      0.218700    0.203240  
min      0.000000    0.000000  
25%      0.000000    1.000000  
50%      0.000000    1.000000  
75%      0.000000    1.000000  
max      1.000000    1.000000

In [38]:
p_test = clf.predict(Xtest)

In [39]:
Xtest.describe()

SibSp       Parch      Classe  sex_female    sex_male       emb_C  \
count  418.000000  418.000000  418.000000  418.000000  418.000000  418.000000   
mean     0.447368    0.392344    0.632775    0.363636    0.636364    0.244019   
std      0.896760    0.981429    0.420919    0.481622    0.481622    0.430019   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    1.000000    0.000000    1.000000    0.000000   
75%      1.000000    0.000000    1.000000    1.000000    1.000000    0.000000   
max      8.000000    9.000000    1.000000    1.000000    1.000000    1.000000   

            emb_Q       emb_S        Prix     Cabin_A     Cabin_B     Cabin_C  \
count  418.000000  418.000000  418.000000  418.000000  418.000000  418.000000   
mean     0.110048    0.645933    0.069373    0.016746    0.043062    0.083732   
std      0.313324    0.478803    0.109046    0.128474    0.203240    0.277317   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.015412    0.000000    0.000000    0.000000   
50%      0.000000    1.000000    0.028213    0.000000    0.000000    0.000000   
75%      0.000000    1.000000    0.061429    0.000000    0.000000    0.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

          Cabin_D     Cabin_E     Cabin_F     Cabin_G     Cabin_X         Age  \
count  418.000000  418.000000  418.000000  418.000000  418.000000  418.000000   
mean     0.031100    0.021531    0.019139    0.002392    0.782297   30.272590   
std      0.173797    0.145320    0.137177    0.048912    0.413179   12.634534   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.170000   
25%      0.000000    0.000000    0.000000    0.000000    1.000000   23.000000   
50%      0.000000    0.000000    0.000000    0.000000    1.000000   30.272590   
75%      0.000000    0.000000    0.000000    0.000000    1.000000   35.750000   
max      1.000000    1.000000    1.000000    1.000000    1.000000   76.000000   

         isMaster      Enfant  
count  418.000000  418.000000  
mean     0.050239    0.956938  
std      0.218700    0.203240  
min      0.000000    0.000000  
25%      0.000000    1.000000  
50%      0.000000    1.000000  
75%      0.000000    1.000000  
max      1.000000    1.000000

In [40]:
result = pd.DataFrame(test['PassengerId'])
pred = pd.DataFrame(p_test, columns=['Survived'])
result = result.join(pred)

In [41]:
result.describe()

PassengerId    Survived
count   418.000000  418.000000
mean   1100.500000    0.433014
std     120.810458    0.496086
min     892.000000    0.000000
25%     996.250000    0.000000
50%    1100.500000    0.000000
75%    1204.750000    1.000000
max    1309.000000    1.000000

In [42]:
result.to_csv("./data/result.csv", columns=["PassengerId", "Survived"], index=False)

# Résultat : score = 0.76555